## 데이터 불러오기

In [4]:
import pymysql

# Establish the connection to the MySQL database
connection = pymysql.connect(
    host="127.0.0.1",    # MySQL host
    user="scott",  # MySQL user
    password="tiger",  # MySQL password
    db="Project",  # MySQL database to connect to
    charset='utf8mb4',   # Charset for encoding
    cursorclass=pymysql.cursors.DictCursor  # To return rows as dictionaries
)

try:
    with connection.cursor() as cursor:
        # Example query: Retrieving data from a table
        sql_query =  
        cursor.execute(sql_query)
        result = cursor.fetchall()
        for row in result:
            print(row)
finally:
    connection.close()  # Make sure to close the connection


SyntaxError: invalid syntax (882206333.py, line 16)

## DML

### insert


In [31]:
sql = "INSERT INTO test (증가량, 계, 년도) VALUES (NULL, 17663188, 2018);"

In [32]:
# connection, cursor 생성 -> with 문으로 작성.
# manual commit 이 default. 
##  DML(insert/update/delete) 처리 후 commit을 실행해야한다.
with pymysql.connect(host="127.0.0.1", port=3306, user="scott", password="tiger", db="Project") as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(sql)
        print("처리 행수:", result)
        # commit 처리
        conn.commit()

처리 행수: 1


### Parameterized Query

In [34]:
import pymysql

# insert할 값 입력받기
증가량 = input("증가량:")
계 = input("계: ")
년도 = input('년도: ')

# SQL 실행
insert_sql = "insert into test (증가량, 계, 년도) values (%s, %s, %s)"

with pymysql.connect(host="127.0.0.1", port=3306, user='scott', password='tiger', db='Project') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(insert_sql, [증가량, 계, 년도])
        conn.commit()
        print("처리 행수:", result)

처리 행수: 1


#### 여러개 행 insert

- executemany() 사용
    - insert할 값들을 가진 리스트를 넣어 한번에 여러 행을 insert한다.


In [39]:
datas = [
    [614847,18686570,2020],
    [481692,19168262,2021],
    [479713,19647975,2022],
    [431319,20079294,2023],
]

In [40]:
insert_sql = "insert into test(증가량, 계, 년도) values(%s, %s, %s)"
with pymysql.connect(host="127.0.0.1", port=3306, user='scott', password='tiger', db='Project') as conn:
    with conn.cursor() as cursor:
        cnt = cursor.executemany(insert_sql, datas)  # executemany() 결과 행수 반환.
        conn.commit()

print("insert된 총 행수:", cnt)

insert된 총 행수: 4


## select (DQL - Data Query Language)

-   조회결과 조회
    -   cursor.execute("select문") 실행 후 cursor의 결과 조회 메소드(fetch메소드)를 이용해 결과를 받는다.
-   fetch메소드
    -   **fetchall()**
        -   조회한 모든 행을을 반환
    -   **fetchmany(size=개수)**
        -   전체 조회한 행들 중 지정한 size개수 만큼 반환.
        -   연속적으로 실행하면 다음 size개수 만큼씩 반환한다.
        -   더 이상 조회한 결과가 없으면 빈 튜플을 반환한다.
    -   **fetchone()**
        -   조회결과 중 첫번째 행만 반환
        -   주로 pk 동등 조건으로 조회한 경우 사용


### fetchall()


In [41]:
sql = "select 증가량, 계, 년도 from test" 
with pymysql.connect(host="127.0.0.1", port=3306, user='scott', password='tiger', db='Project') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(sql)
        print("조회행수:", result)
        resultset = cursor.fetchall()

조회행수: 6


In [42]:
resultset

((None, 17663188, 2018),
 (408535, 18071723, 2019),
 (614847, 18686570, 2020),
 (481692, 19168262, 2021),
 (479713, 19647975, 2022),
 (431319, 20079294, 2023))

### 조회결과를 dictionary로 반환
- pymysql.cursors.DictCursor 사용
    - Connection 생성시 또는 Cursor 생성시 지정한다.
- key: 컬럼명, value: 컬럼값

In [45]:
sql = "select 증가량, 계, 년도 from test" 
with pymysql.connect(host="127.0.0.1", port=3306, user='scott', password='tiger', db='Project') as conn:
    with conn.cursor(pymysql.cursors.DictCursor) as cursor:  
        result = cursor.execute(sql)
        print("조회행수:", result)
        resultset = cursor.fetchall()

조회행수: 6


In [46]:
resultset

[{'증가량': None, '계': 17663188, '년도': 2018},
 {'증가량': 408535, '계': 18071723, '년도': 2019},
 {'증가량': 614847, '계': 18686570, '년도': 2020},
 {'증가량': 481692, '계': 19168262, '년도': 2021},
 {'증가량': 479713, '계': 19647975, '년도': 2022},
 {'증가량': 431319, '계': 20079294, '년도': 2023}]

### fetchone()


In [53]:
sql = "select * from test where 년도 = %s" 
with pymysql.connect(host="127.0.0.1", port=3306, user='scott', password='tiger', db='Project') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(sql, [2018]) # []안에 조회할 id(index)
        print("조회행수:", result)
        resultset = cursor.fetchone()

조회행수: 1


In [51]:
resultset

(None, 17663188, 2018)

### fetchmany()


In [55]:
sql = "select 증가량, 계, 년도 from test" 
with pymysql.connect(host="127.0.0.1", port=3306, user='scott', password='tiger', db='Project') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(sql)
        print("조회행수:", result)
        resultset1 = cursor.fetchmany(size=2)  # 처음 두개
        resultset2 = cursor.fetchmany(size=2)  # 다음 두개
        resultset3 = cursor.fetchmany(size=2)
        resultset4 = cursor.fetchmany(size=2)

조회행수: 6


In [56]:
print(resultset1)
print(resultset2)
print(resultset3)
print(resultset4)

((None, 17663188, 2018), (408535, 18071723, 2019))
((614847, 18686570, 2020), (481692, 19168262, 2021))
((479713, 19647975, 2022), (431319, 20079294, 2023))
()


### cursor 는 iterable 타입

-   for in 문에 select query를 실행한 cursor를 사용하면 조회결과를 한 행씩 조회할 수 있다.


In [7]:
sql = "select * from test" 
with pymysql.connect(host="127.0.0.1", port=3306, user='scott', password='tiger', db='Project') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(sql)
        for 증가량, 계, 년도 in cursor:
            print(증가량, 계, 년도, sep=" , ")

None , 17663188 , 2018
408535 , 18071723 , 2019
614847 , 18686570 , 2020
481692 , 19168262 , 2021
479713 , 19647975 , 2022
431319 , 20079294 , 2023
